## Download and preprocess raw data
The data used in this document was obtained from the Kaggle competition Addison Howard, Chris Balbach, Clayton Miller, Jeff Haberl, Krishnan Gowri, Sohier Dane. (2019). ASHRAE - Great Energy Predictor III. Kaggle. Raw data are available for download after registration at Kaggle platform.

Three files are used:

- These files contain weather-related data for each building, including outside air temperature, dew point temperature, relative humidity and other weather parameters. The weather data is critical for understanding the impact of external conditions on building energy use.

- building_metadata.csv: This file contains metadata for each building in the dataset, such as building type, primary use, square footage, number of floors, and year built. This information helps to understand the characteristics of the buildings and their potential influence on energy consumption patterns.

- train.csv: The train dataset contains the target variable, i.e. the energy consumption data for each building, together with the time stamps for the energy consumption readings. It also includes the corresponding building and weather IDs to link the information across different datasets.

Only buildings with more than 85% of values different than NaN or Zero are selected for this study. Among the different types of energy consumption measures, only the electricity meter is used. Finally, the data is aggregated to a daily frequency.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

In [2]:
# Path to files
data_dir = Path("./data/raw/")

# Building metadata ("building_metadata.csv")
building_metadata_path = data_dir / "building_metadata.csv"

# Energy consumption ("train.csv")
train_path = data_dir / "train.csv"

# Weather ("weather_train.csv")
weather_train_path = data_dir / "weather_train.csv"


We will specify `dtypes` of each column and pass this to `pd.read_csv` when loading data. This will allow us to be more memory efficient (e.g., using `category` rather than `object` to denote categorical variables in our pandas dataframes).

In [3]:
# Open the raw data from Kaggle
data_dir = Path("./data/raw/")

# Building metadata
building_metadata_dtypes = {
    "site_id": pd.Int32Dtype(),
    "building_id": pd.Int32Dtype(),
    "primary_use": "category",
    "square_feet": pd.Int32Dtype(),
    "year_built": pd.Int32Dtype(),
    "floor_count": pd.Int32Dtype(),
}

building_metadata = pd.read_csv(building_metadata_path, dtype=building_metadata_dtypes)
print("Building metadata shape:", building_metadata.shape)
display(building_metadata.head(3))

# Consumption data
train_dtypes = {
    "building_id": pd.Int32Dtype(),
    "meter": pd.Int16Dtype(),
    "meter_reading": np.float32,
}

train = pd.read_csv(train_path, dtype=train_dtypes)
train["timestamp"] = pd.to_datetime(train["timestamp"])
print("Consumption data shape:", train.shape)
display(train.head(3))

# Weather data
weather_dtypes = {
    "site_id": pd.Int32Dtype(),
    "air_temperature" : np.float64,
    "cloud_coverage" : np.float64,
    "dew_temperature" : np.float64,
    "precip_depth_1_hr" : np.float64,
    "sea_level_pressure" : np.float64,
    "wind_direction" : np.float64,
    "wind_speed" : np.float64,
}

weather_train = pd.read_csv(weather_train_path, dtype=weather_dtypes)
weather_train["timestamp"] = pd.to_datetime(weather_train["timestamp"])
print("Weather data shape:", weather_train.shape)
display(weather_train.head(3))

Building metadata shape: (1449, 6)


,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008,<NA>
1,0,1,Education,2720,2004,<NA>
2,0,2,Education,5376,1991,<NA>


Consumption data shape: (20216100, 4)


,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0


Weather data shape: (139773, 9)


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0


In [4]:
print("")
print("Size of data sets before filtering:")
print("==================================")
print(f"Number of rows in train: {train.shape[0]}")
print(f"Number of rows in building_metadata: {building_metadata.shape[0]}")
print(f"Number of rows in weather: {weather_train.shape[0]}")
print(f"Number of buildings: {train['building_id'].nunique()}")

# The variable meter indicates the type of meter that measures energy consumption. There
# are 4 types of meter: {0: electricity, 1: chilled water, 2: steam, 3: hot water}. Only
# the electricity meter, which is the most common, is selected.
train_processed = (
       train.query("meter == 0") # Filter to electricity meter only
            .drop(columns=["meter"]) # Drop the meter column
       )


Size of data sets before filtering:
Number of rows in train: 20216100
Number of rows in building_metadata: 1449
Number of rows in weather: 139773
Number of buildings: 1449


In [5]:
# Now we resample the frequency of each time series to be hourly. This will
# introduce rows of missing values for `meter_reading` where there are 
# missing timestamps. 

# Explicitly introducing these missing values allows to analyse how much
# missing data there is due to missing timestamps.

train_processed = train_processed.groupby("building_id").apply(
    lambda df: df.set_index("timestamp").loc[:, ["meter_reading"]].asfreq("H")
).reset_index()

In [6]:
# We will remove buildings which have larger than 85% of missing observations
# or recordings of zero.
percentage_missing_or_zero_by_building = train_processed.groupby("building_id").apply(
    lambda df: (df["meter_reading"].isnull() | df["meter_reading"] == 0).mean()
)

# Filter out buildings with large amounts of missing observations or zeros.
selected_buildings = percentage_missing_or_zero_by_building[
    percentage_missing_or_zero_by_building < 0.15
].index

In [7]:
# Exclude information about buildings that were not selected
building_metadata_processed = building_metadata[
    building_metadata["building_id"].isin(selected_buildings)
]
train_processed = train_processed[
    train_processed["building_id"].isin(selected_buildings)
]
selected_sites = building_metadata_processed["site_id"].unique()
weather_train_processed = weather_train[weather_train["site_id"].isin(selected_sites)]


print("")
print("Size of data sets after filtering:")
print("==================================")
print(f"Number of rows in train: {train_processed.shape[0]}")
print(f"Number of rows in building_metadata: {building_metadata_processed.shape[0]}")
print(f"Number of rows in weather: {weather_train_processed.shape[0]}")
print(f"Number of buildings: {train_processed['building_id'].nunique()}")


Size of data sets after filtering:
Number of rows in train: 11130347
Number of rows in building_metadata: 1277
Number of rows in weather: 139773
Number of buildings: 1277


In [8]:
# Group data from hourly to daily
train_daily = (
    train_processed.groupby(["building_id", pd.Grouper(key="timestamp", freq="D")])[
        "meter_reading"
    ]
    .sum()
    .reset_index()
    .sort_values(["building_id", "timestamp"])
)
display(train_daily.head(3))

weather_train_daily = (
    weather_train_processed.groupby(["site_id", pd.Grouper(key="timestamp", freq="D")])
    .mean()
    .reset_index()
    .sort_values(["site_id", "timestamp"])
)
display(weather_train_daily.head(3))

,building_id,timestamp,meter_reading
0,29,2016-01-06,0.000000
1,29,2016-01-07,577.445984
2,29,2016-01-08,0.000000


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01,23.337500,4.285714,20.020833,-0.173913,1018.926087,122.916667,1.854167
1,0,2016-01-02,19.537500,5.666667,15.325000,-0.083333,1018.960870,167.500000,3.925000
2,0,2016-01-03,14.829167,8.000000,12.479167,0.500000,1017.241667,276.666667,5.000000


In [11]:
# Join data
data = pd.merge(train_daily, building_metadata_processed, on="building_id", how="left")
data = pd.merge(data, weather_train_daily, on=["site_id", "timestamp"], how="left")
data = data.sort_values(["site_id", "building_id", "timestamp" ])

# Fill nan values in temperature and wind speed with forward fill
data['air_temperature'] = data['air_temperature'].ffill()
data['wind_speed'] = data['wind_speed'].ffill()

In [12]:
data.to_parquet(path="./data/processed/data.parquet")